# Modelo Random Forest para Predicción de Rendimiento

Este notebook carga los datos de `train.csv` y `test.csv`, preprocesa la información y entrena un modelo Random Forest para generar el archivo de sumisión.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

# 1. Cargar datos
print("Cargando datos...")
try:
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    print("Datos cargados exitosamente.")
    print("Train shape:", train.shape)
    print("Test shape:", test.shape)
except FileNotFoundError:
    print("ERROR: No se encontraron los archivos 'train.csv' o 'test.csv'.")
    print("Por favor, súbelos a la carpeta de archivos de Colab (panel izquierdo).")

Cargando datos...
ERROR: No se encontraron los archivos 'train.csv' o 'test.csv'.
Por favor, súbelos a la carpeta de archivos de Colab (panel izquierdo).


In [ ]:
# 2. Preprocesamiento

target_col = 'RENDIMIENTO_GLOBAL'
id_col = 'ID'

# Separar target y features
y = train[target_col]
X = train.drop([target_col, id_col], axis=1)
X_test = test.drop([id_col], axis=1)

# Identificar columnas
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

print(f"Variables numéricas: {len(numeric_cols)}")
print(f"Variables categóricas: {len(categorical_cols)}")

# Imputación (Rellenar nulos)
# Numéricas -> Mediana
imputer_num = SimpleImputer(strategy='median')
X[numeric_cols] = imputer_num.fit_transform(X[numeric_cols])
X_test[numeric_cols] = imputer_num.transform(X_test[numeric_cols])

# Categóricas -> Moda
imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = imputer_cat.fit_transform(X[categorical_cols])
X_test[categorical_cols] = imputer_cat.transform(X_test[categorical_cols])

# Codificación (Texto a Números)
print("Codificando variables categóricas...")
for col in categorical_cols:
    le = LabelEncoder()
    # Combinar para ajustar el encoder con todas las categorías posibles
    full_data = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(full_data)

    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

print("Preprocesamiento completado.")

Variables numéricas: 5
Variables categóricas: 14
Codificando variables categóricas...
Preprocesamiento completado.


In [ ]:
# 3. Entrenamiento y Validación Local

# Separamos un 20% de train para validar que el modelo funciona bien
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X, y, test_size=0.2, random_state=42)

print("Entrenando Random Forest (validación)...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train_split, y_train_split)

val_preds = rf.predict(X_val_split)
acc = accuracy_score(y_val_split, val_preds)
print(f"Accuracy en validación: {acc:.4f}")
print("\nReporte de Clasificación:\n", classification_report(y_val_split, val_preds))

Entrenando Random Forest (validación)...
Accuracy en validación: 0.4014

Reporte de Clasificación:
               precision    recall  f1-score   support

        alto       0.51      0.59      0.55     35165
        bajo       0.42      0.51      0.46     34573
  medio-alto       0.30      0.24      0.27     34259
  medio-bajo       0.31      0.26      0.29     34503

    accuracy                           0.40    138500
   macro avg       0.39      0.40      0.39    138500
weighted avg       0.39      0.40      0.39    138500



In [ ]:
# 4. Generar Predicciones Finales y Submission

print("Re-entrenando con TODOS los datos...")
rf_full = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_full.fit(X, y)

print("Generando predicciones para test.csv...")
test_preds = rf_full.predict(X_test)

submission = pd.DataFrame({
    'ID': test[id_col],
    'RENDIMIENTO_GLOBAL': test_preds
})

submission.to_csv('submission.csv', index=False)
print("¡Listo! Archivo 'submission.csv' generado.")
print("Descárgalo desde el panel de archivos de la izquierda.")

Re-entrenando con TODOS los datos...
Generando predicciones para test.csv...
¡Listo! Archivo 'submission.csv' generado.
Descárgalo desde el panel de archivos de la izquierda.
